In [25]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
from semantic_metrics import *
import pandas as pd
import time
import os

load_dotenv()

True

In [38]:
DOCS = ["nvidia.pdf", "mastercard.pdf", "microsoft.pdf"]
# mistal-tiny = 7B and mistral-small = 8x7B
MISTRAL_MODELS = ["mistral-tiny", "mistral-small", "mistral-large-2402"]
MISTRAL_EMBEDDINGS = "mistral-embed"
OPENAI_MODELS = ["gpt-4-0125-preview", "gpt-3.5-turbo"]
OPENAI_EMBEDDINGS = "text-embedding-3-large"
GOOGLE_MODELS = ['gemini-1.0-pro-001']
GOOGLE_EMBEDDINGS = 'embedding-001'
# LLM settings
TEMPERATURE = 0.1
TOP_P = 0.2

In [39]:
def run_mistral_completions(questions, reference_doc, llm_model, embeddings_model, temperature=0.5, top_p=0.5):

    # Load data
    loader = PDFPlumberLoader(reference_doc)
    docs = loader.load()
    # Split text into chunks 
    text_splitter = RecursiveCharacterTextSplitter()
    documents = text_splitter.split_documents(docs)
    # Define the embedding model
    embeddings = MistralAIEmbeddings(model=embeddings_model, mistral_api_key=os.getenv("MISTRAL_API_KEY"))
    # Create the vector store 
    vector = FAISS.from_documents(documents, embeddings)
    # Define a retriever interface
    retriever = vector.as_retriever()
    # Define prompt template
    prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

    <context>
    {context}
    </context>

    Question: {input}""")

    # Define LLM
    model = ChatMistralAI(mistral_api_key=os.getenv("MISTRAL_API_KEY"), model=llm_model, temperature=temperature, top_p=top_p)

    # Create a retrieval chain to answer questions
    document_chain = create_stuff_documents_chain(model, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    answers = []
    times = []
    for question in questions:
        start = time.time()
        response = retrieval_chain.invoke({"input": question})
        total_time = time.time() - start
        answer = response["answer"]
        answers.append(answer)
        times.append(total_time)

    return answers, times

def run_openai_completions(questions, reference_doc, llm_model, embeddings_model, temperature=0.5, top_p=0.5):

    # Load data
    loader = PDFPlumberLoader(reference_doc)
    docs = loader.load()
    # Split text into chunks 
    text_splitter = RecursiveCharacterTextSplitter()
    documents = text_splitter.split_documents(docs)
    # Define the embedding model
    embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"), model=embeddings_model)
    # Create the vector store 
    vector = FAISS.from_documents(documents, embeddings)
    # Define a retriever interface
    retriever = vector.as_retriever()
    # Define prompt template
    prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

    <context>
    {context}
    </context>

    Question: {input}""")

    # Define LLM
    model = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"), model_name=llm_model, temperature=temperature, model_kwargs={"top_p": top_p})

    # Create a retrieval chain to answer questions
    document_chain = create_stuff_documents_chain(model, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    answers = []
    times = []
    for question in questions:
        start = time.time()
        response = retrieval_chain.invoke({"input": question})
        total_time = time.time() - start
        answer = response["answer"]
        answers.append(answer)
        times.append(total_time)

    return answers, times

def run_google_completions(questions, reference_doc, llm_model, embeddings_model, temperature=0.5, top_p=0.5):

    # Load data
    loader = PDFPlumberLoader(reference_doc)
    docs = loader.load()
    # Split text into chunks 
    text_splitter = RecursiveCharacterTextSplitter()
    documents = text_splitter.split_documents(docs)
    # Define the embedding model
    embeddings = GoogleGenerativeAIEmbeddings(model=f'models/{embeddings_model}')
    # Create the vector store 
    vector = FAISS.from_documents(documents, embeddings)
    # Define a retriever interface
    retriever = vector.as_retriever()
    # Define prompt template
    prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

    <context>
    {context}
    </context>

    Question: {input}""")

    # Define LLM
    model = ChatGoogleGenerativeAI(
        google_api_key=os.getenv("GOOGLE_API_KEY"),
        model=f'models/{llm_model}',
        temperature=temperature,
        top_p=top_p
    )

    # Create a retrieval chain to answer questions
    document_chain = create_stuff_documents_chain(model, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    answers = []
    times = []
    for question in questions:
        start = time.time()
        response = retrieval_chain.invoke({"input": question})
        total_time = time.time() - start
        answer = response["answer"]
        answers.append(answer)
        times.append(total_time)

    return answers, times

In [48]:
# Load set of questions
questions_df = pd.read_csv('questions.csv')

df_list = []

# Run completions
for doc in DOCS:
    print(doc)
    print()
    doc_df = questions_df[questions_df['ReferenceDoc'] == doc]
    doc_path = f'docs/{doc}'
    for model in MISTRAL_MODELS:
        print(model)
        mistral_answers, mistral_times = run_mistral_completions(
            doc_df['Question'].to_list(),
            doc_path,
            llm_model=model,
            embeddings_model=MISTRAL_EMBEDDINGS,
            temperature=TEMPERATURE,
            top_p=TOP_P
        )
        doc_df[f'{model}_response'] = mistral_answers
        doc_df[f'{model}_time'] = mistral_times

    for model in OPENAI_MODELS:
        print(model)
        openai_answers, openai_times = run_openai_completions(
            doc_df['Question'].to_list(),
            doc_path,
            llm_model=model,
            embeddings_model=OPENAI_EMBEDDINGS,
            temperature=TEMPERATURE,
            top_p=TOP_P
        )
        doc_df[f'{model}_response'] = openai_answers
        doc_df[f'{model}_time'] = openai_times

    for model in GOOGLE_MODELS:
        print(model)
        google_answers, google_times = run_google_completions(
            doc_df['Question'].to_list(),
            doc_path,
            llm_model=model,
            embeddings_model=GOOGLE_EMBEDDINGS,
            temperature=TEMPERATURE,
            top_p=TOP_P
        )
        doc_df[f'{model}_response'] = google_answers
        doc_df[f'{model}_time'] = google_times
    
    df_list.append(doc_df)

responses_df = pd.concat(df_list)

nvidia.pdf

gemini-1.0-pro-001


/var/folders/tf/rp1vqsws5nq6gh7ndddbtl1h0000gn/T/ipykernel_39776/514868342.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df[f'{model}_response'] = google_answers
/var/folders/tf/rp1vqsws5nq6gh7ndddbtl1h0000gn/T/ipykernel_39776/514868342.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df[f'{model}_time'] = google_times


mastercard.pdf

gemini-1.0-pro-001


/var/folders/tf/rp1vqsws5nq6gh7ndddbtl1h0000gn/T/ipykernel_39776/514868342.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df[f'{model}_response'] = google_answers
/var/folders/tf/rp1vqsws5nq6gh7ndddbtl1h0000gn/T/ipykernel_39776/514868342.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df[f'{model}_time'] = google_times


microsoft.pdf

gemini-1.0-pro-001


/var/folders/tf/rp1vqsws5nq6gh7ndddbtl1h0000gn/T/ipykernel_39776/514868342.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df[f'{model}_response'] = google_answers
/var/folders/tf/rp1vqsws5nq6gh7ndddbtl1h0000gn/T/ipykernel_39776/514868342.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doc_df[f'{model}_time'] = google_times


In [49]:
responses_df

,Question,Answer,ReferenceDoc,gemini-1.0-pro-001_response,gemini-1.0-pro-001_time
0,What was Nvidia's total revenue for the quarte...,Nvidia's total revenue for the quarter ended A...,nvidia.pdf,"$7,192 million",12.240479
1,What are the primary sources of Nvidia's reven...,"For the quarter ended April 30, 2023, Nvidia's...",nvidia.pdf,The primary sources of Nvidia's revenue for th...,3.429274
2,What were Nvidia's total operating expenses fo...,"For the quarter ended April 30, 2023, Nvidia's...",nvidia.pdf,Nvidia's total operating expenses for the quar...,2.843790
3,Were there any significant changes in Nvidia's...,The COGS as a percentage of revenue increased ...,nvidia.pdf,The provided context does not contain informat...,2.530924
4,What is Nvidia's net income for the quarter en...,Nvidia's net income for the quarter ended Apri...,nvidia.pdf,"$2,043 million",1.843221
5,What are the company's gross profit margin and...,"For the quarter ended April 30, 2023, Nvidia's...",nvidia.pdf,The provided context does not include informat...,2.457596
6,What was Nvidia's current ratio (current asset...,"As of April 30, 2023, Nvidia's current assets ...",nvidia.pdf,Nvidia's current ratio for the quarter ended A...,2.867827
7,Was there a significant change in Nvidia's cas...,"Yes, there was a significant change in Nvidia'...",nvidia.pdf,"Yes, there was a significant change in Nvidia'...",2.641461
8,What was Nvidia's debt-to-equity ratio for the...,0.81,nvidia.pdf,The provided context does not include informat...,1.837309
9,Were there any significant new debts or financ...,Nvidia reported outstanding inventory purchase...,nvidia.pdf,The provided context does not mention any sign...,2.893212


In [50]:
# Evaluate responses
for model in MISTRAL_MODELS + OPENAI_MODELS + GOOGLE_MODELS:
    responses_df[f'{model}_cosine_sim'] = eval_cosine_similarity(
        responses_df['Answer'].to_list(),
        responses_df[f'{model}_response'].to_list(),
        model = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
    responses_df[f'{model}_bertscore'] = bertscores(
        responses_df['Answer'].to_list(),
        responses_df[f'{model}_response'].to_list())
    token_recall, token_precision = token_recall_precision(
        responses_df['Answer'].to_list(),
        responses_df[f'{model}_response'].to_list())
    responses_df[f'{model}_token_recall'] = token_recall
    responses_df[f'{model}_token_precision'] = token_precision
    responses_df[f'{model}_gpt_evaluation'] = gpt_evaluation(
        responses_df['Question'].to_list(),
        responses_df['Answer'].to_list(),
        responses_df[f'{model}_response'].to_list(),
        key=os.getenv('OPENAI_API_KEY'))

In [52]:
responses_df.to_csv('responses_df.csv')